In [1]:
import os
import pandas as pd
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from tagger.core.mongo.models.order import Order, OrderType, OrderCodeMap, Payment
from tagger.models.inventory import Inventory, InventoryStatus
from tagger.core.label.print_label import print_label
from tagger.core.label.shipping_label import make_shipping_label
from bson import ObjectId

In [19]:
orders = (
            Order.objects(
                orderstatus__in=[
                    "PAYMENT_FINISHED",
                    "PRODUCT_PREPARING",
                    "DELIVERY_PREPARING",
                ],
            )
            .order_by("id")
            .all()
        )

In [20]:
for order in orders:
    payment = order.get_payment()
    
    print("---------------------------------------------------------------------------")
    print(order.id, order.orderstatus, payment.buyername, payment.name, order.created)
    for item in order.orders:
        inv_no = ""
        if item.alloffproduct:
            print("TIMEDEAL", item.alloffproduct.name, item.alloffproduct._id, item.size, item.quantity)
        if item.product:
            print("NORMAL", item.product.name, item.product._id, item.size, item.quantity)

---------------------------------------------------------------------------
6173e73f17fc2e5a1f9e9e88 DELIVERY_PREPARING 권재희 네이비 체크 셔츠 2021-10-23 10:43:11.561000
TIMEDEAL 네이비 체크 셔츠 6173d7abee2c39d09bebea5e M 1
---------------------------------------------------------------------------
61740dea17fc2e5a1f9e9f1c DELIVERY_PREPARING 권미영 캐시미어 롱 가디건(블랙) 2021-10-23 13:28:10.053000
TIMEDEAL 캐시미어 롱 가디건(블랙) 6173d9d8ee2c39d09bebea62 S 1
---------------------------------------------------------------------------
617533f017fc2e5a1f9ea0da DELIVERY_PREPARING 박혜란 [택 제거 상품] BETTA(Brown) 2021-10-24 10:22:40.992000
TIMEDEAL [택 제거 상품] BETTA(Brown) 6173b70bee2c39d09bebea56 38 1
---------------------------------------------------------------------------
6175340117fc2e5a1f9ea0dd DELIVERY_PREPARING 서지영 [택 제거 상품] JULIA(Black) 2021-10-24 10:22:57.104000
TIMEDEAL [택 제거 상품] JULIA(Black) 6173b8efee2c39d09bebea59 38 1
---------------------------------------------------------------------------
61755cd317fc2e5a1f9ea14e

---------------------------------------------------------------------------
618139cf26685ff051754d2a DELIVERY_PREPARING 김태연 백 플랩 니트 드레스(OTMEAL) 2021-11-02 13:14:55.247000
TIMEDEAL 백 플랩 니트 드레스(OTMEAL) 617f6cd4ee2c39d09bebebc0 90 1
---------------------------------------------------------------------------
6181e47a26685ff051754e97 DELIVERY_PREPARING 우정현 벨티드 후드 코트(멜란지 베이지)  2021-11-03 01:23:06.574000
TIMEDEAL 벨티드 후드 코트(멜란지 베이지)  6178b514ee2c39d09bebeb23 88(66) 1
---------------------------------------------------------------------------
6181f47c26685ff051754eb2 DELIVERY_PREPARING 노은숙 VISIDOR 트위드 자켓 2021-11-03 02:31:24.346000
TIMEDEAL VISIDOR 트위드 자켓 6177fc6bee2c39d09bebeb0a 36 1
---------------------------------------------------------------------------
618265fc26685ff051754f33 DELIVERY_PREPARING 진혜수 핸드메이드 캐시미어 코트  2021-11-03 10:35:40.355000
TIMEDEAL 핸드메이드 캐시미어 코트  617ca47bee2c39d09bebeba1 85 1
---------------------------------------------------------------------------
618267a226685ff05175

---------------------------------------------------------------------------
6185461526685ff051755590 DELIVERY_PREPARING 오기 [빈폴스포츠] 리브 퀄팅 구스 다운 점퍼(화이트)  2021-11-05 14:56:21.537000
TIMEDEAL [빈폴스포츠] 리브 퀄팅 구스 다운 점퍼(화이트)  6184fc79ee2c39d09bebec4c 95 1
---------------------------------------------------------------------------
61854bb026685ff0517555ab DELIVERY_PREPARING 이강미 케이블 블렌드 니트 탑  2021-11-05 15:20:16.349000
TIMEDEAL 케이블 블렌드 니트 탑  6180f0f7ee2c39d09bebec06 90(F) 1
---------------------------------------------------------------------------
6185ace326685ff051755600 DELIVERY_PREPARING 정나영 퀄팅 구스다운 점퍼(BLACK)  2021-11-05 22:14:59.564000
TIMEDEAL 퀄팅 구스다운 점퍼(BLACK)  6180d45fee2c39d09bebebd6 88(66) 1
---------------------------------------------------------------------------
6185bcaf26685ff051755618 DELIVERY_PREPARING 고유정 울 캐시미어 핸드메이드 벨티드 코트_TWHCTJ80040 2021-11-05 23:22:23.452000
NORMAL 울 캐시미어 핸드메이드 벨티드 코트_TWHCTJ80040 60bdbc3d881252a2bd32f911 Dark Navy - 55 1
------------------------------------

---------------------------------------------------------------------------
6188f7cd26685ff051755b36 PAYMENT_FINISHED 박은진  [빈폴스포츠] 후드 폭스 다운 점퍼  2021-11-08 10:11:25.092000
TIMEDEAL  [빈폴스포츠] 후드 폭스 다운 점퍼  6188d127ee2c39d09bebec71 100 1
---------------------------------------------------------------------------
6188fbf826685ff051755b45 PAYMENT_FINISHED 조혜은 [빈폴스포츠] (MEN) 올라운더 보아 구스 롱 다운 2021-11-08 10:29:12.311000
TIMEDEAL [빈폴스포츠] (MEN) 올라운더 보아 구스 롱 다운 6184f8a4ee2c39d09bebec4a 100 1
---------------------------------------------------------------------------
6188fe8526685ff051755b4d PAYMENT_FINISHED 유정연 [빈폴스포츠] (MEN) 롱 구스 다운 점퍼  2021-11-08 10:40:05.384000
TIMEDEAL [빈폴스포츠] (MEN) 롱 구스 다운 점퍼  6188c43cee2c39d09bebec6b 110 1
---------------------------------------------------------------------------
6188fecf26685ff051755b4f PAYMENT_FINISHED 권은진  [빈폴스포츠] 푸퍼 다운 점퍼  2021-11-08 10:41:19.014000
TIMEDEAL  [빈폴스포츠] 푸퍼 다운 점퍼  6188cdd5ee2c39d09bebec70 95 1
---------------------------------------------------

In [ ]:
inv_keys.values()

In [21]:
inventories = Inventory.objects.filter(status=InventoryStatus.IN_STOCK).all()
inv_keys = {}
for inv in inventories:
    inv_keys[inv.id] = inv.product_name.replace(" ", "") + "-" + inv.product_brand
    
possible_order_ids = []
exhausted_inventory_nos = []
for order in orders:
    payment = order.get_payment()
    
    possible = True
    inv_nos = []
    for item in order.orders:
        inv_no = ""
        if item.alloffproduct:
            invs = Inventory.objects.filter(product_id=item.alloffproduct._id, status=InventoryStatus.IN_STOCK).all()
            for inv in invs:
                if inv.size == item.size:
                    inv_no = f"{inv.code} ({str(inv.id)})"
                    continue
            
            item_key = item.alloffproduct.name.replace(" ", "") + "-" + item.alloffproduct.brand.keyname

            if item_key in inv_keys.values():
                for key in inv_keys:
                    if inv_keys[key] == item_key:
                        inv = Inventory.objects.get(id=key)
                        if inv.size == item.size:
                            inv_no = f"{inv.code} ({str(inv.id)})"
                            continue
                            
                    
        if item.product:
            invs = Inventory.objects.filter(product_id=item.product._id, status=InventoryStatus.IN_STOCK).all()
            for inv in invs:
                if inv.size == item.size:
                    inv_no = f"{inv.code} ({str(inv.id)})"
                    continue
            
            item_key = item.product.name.replace(" ", "") + "-" + item.product.brand.keyname

            if item_key in inv_keys.values():
                for key in inv_keys:
                    if inv_keys[key] == item_key:
                        inv = Inventory.objects.get(id=key)
                        if inv.size == item.size:
                            inv_no = f"{inv.code} ({str(inv.id)})"
                            continue
                            
        if inv_no != "" and inv_no not in exhausted_inventory_nos:
            inv_nos.append(inv_no)
            exhausted_inventory_nos.append(inv_no)
        else:
            possible = False

    if possible:
        possible_order_ids.append(order.id)
        print("---------------------------------------------------------------------------")
        print(order.id, order.orderstatus, payment.buyername, payment.name, order.created)
        print("INV_NOS", inv_nos)
    

In [6]:
possible_order_ids

[]

In [10]:
def ready_for_shipping(order_id: str, inventory: Inventory):
    order = Order.objects(id=ObjectId(order_id)).first()
    inventory.out_order_id = order_id
    inventory.status = InventoryStatus.SHIPPING_PENDING
    inventory.save()
    print_label(make_shipping_label(order))

In [11]:
ready_for_shipping("618263b826685ff051754f24",
                   Inventory.objects.get(code="211108-005")
                  )

In [20]:
ready_for_shipping("616bfc4c17fc2e5a1f9e8d24",
                   Inventory.objects.get(id=86)
                  )

In [2]:
to_be_shipped = Inventory.objects.filter(status=InventoryStatus.SHIPPING_PENDING).all()

In [7]:
df = pd.DataFrame([{
    "order_id": x.out_order_id,
    "name": x.out_order.get_payment().buyername,
    "order_name": x.out_order.get_payment().name,
    "inventory_code": x.code
} for x in to_be_shipped])

In [8]:
df.to_clipboard()

In [9]:
Inventory.objects.filter(out_order_id="6188765726685ff051755a57").first().id

70